In [1]:
import pandas as pd
import os
import plotly.express as px

# Preparing Data

In [2]:
# Concatenate all files

def concat_un_comtrade(raw_folder, csv_folder):
    '''
    concat un comtrade
    '''
    df = pd.DataFrame()
    dir_list = os.listdir(raw_folder)
    for file in dir_list:
        path = raw_folder + file
        un_comtrade = pd.read_csv(path,
                                  usecols = ['yr', 'rtCode', 'rtTitle', 
                                             'rt3ISO', 'ptCode', 'ptTitle', 
                                             'pt3ISO', 'cmdCode', 
                                             'cmdDescE', 'TradeValue'],
                                  dtype = {'rtCode': 'str', 'ptCode': 'str', 
                                           'cmdCode': 'str'})
        df = pd.concat([df, un_comtrade])
    
    # create csv file
    filename = csv_folder + "un_comtrade_top30" + ".csv"
    df.to_csv(filename, index = False)
        

In [3]:
raw_folder = 'rawdata/uncomtrade/top30/un_comtrade/'
csv_folder = 'rawdata/'

concat_un_comtrade(raw_folder, csv_folder)

In [4]:
# Read all necessary data
un_comtrade = pd.read_csv('rawdata/un_comtrade_top30.csv',
                                  dtype = {'rtCode': 'str', 'ptCode': 'str', 
                                           'cmdCode': 'str'})
owid_covid = pd.read_csv('rawdata/owid_covid_data.csv',
                         usecols = ['iso_code', 'date', 'total_cases', 
                                    'new_cases', 'stringency_index'])          

In [5]:
un_comtrade.head()

,yr,rtCode,rtTitle,rt3ISO,ptCode,ptTitle,pt3ISO,cmdCode,cmdDescE,TradeValue
0,2020,36,Australia,AUS,0,World,WLD,01,Animals; live,1342439314
1,2020,36,Australia,AUS,32,Argentina,ARG,01,Animals; live,58309
2,2020,36,Australia,AUS,48,Bahrain,BHR,01,Animals; live,4527
3,2020,36,Australia,AUS,96,Brunei Darussalam,BRN,01,Animals; live,8423286
4,2020,36,Australia,AUS,104,Myanmar,MMR,01,Animals; live,311777


## Clean OWID data

In [6]:
# Clean owid data
owid_covid['year'] = owid_covid['date'].str[:4]
owid_covid = owid_covid.loc[:, owid_covid.columns != 'date']
owid_covid.head()

,iso_code,total_cases,new_cases,stringency_index,year
0,AFG,5.0,5.0,8.33,2020
1,AFG,5.0,0.0,8.33,2020
2,AFG,5.0,0.0,8.33,2020
3,AFG,5.0,0.0,8.33,2020
4,AFG,5.0,0.0,8.33,2020


In [7]:
# Group By
agg = {'total_cases': 'last', 'new_cases': 'sum', 
       'stringency_index': 'mean'}
grouped = owid_covid.groupby(['iso_code', 'year']).agg(agg) \
          .reset_index()
owid_df = grouped[grouped['year'] == '2020']

owid_df.head()

,iso_code,year,total_cases,new_cases,stringency_index
0,ABW,2020,5489.0,5489.0,53.860102
3,AFG,2020,52330.0,52330.0,47.626442
6,AGO,2020,17553.0,17553.0,73.365645
9,AIA,2020,13.0,13.0,NaN
12,ALB,2020,58316.0,58316.0,64.760836


#### WTO Merchandise Export

In [8]:
wto_export = pd.read_csv("rawdata/merchandise_values_annual_dataset.csv")
wto_export = wto_export[wto_export['Indicator'] == 'Export']
wto_export = wto_export[wto_export['ProductCode'] == 'TO']
wto_export.head()

,Indicator,ReporterCode,ReporterISO3A,Reporter,ProductCode,Product,Year,Value
12634,Export,662,LCA,Saint Lucia,TO,Total merchandise,2020,55
12635,Export,276,DEU,Germany,TO,Total merchandise,2020,1380647
12636,Export,388,JAM,Jamaica,TO,Total merchandise,2020,1219
12637,Export,LMX,NaN,LDC exporters of manufactures,TO,Total merchandise,2020,55273
12638,Export,666,SPM,Saint Pierre and Miquelon,TO,Total merchandise,2020,5


# Analysis

## First Question: Ranking

## Filter for top countries

We ranked which country performs best or worst during the pandemic

In [9]:
import get_data
from IPython.display import display

top30_countries = pd.read_json('rawdata/reporterAreas_top30.json')
country_list = top30_countries['text']

top30_countries['iso3a'] = get_data.find_iso3a(country_list)
top30_countries.iloc[5,2] = 'HKG'
top30_countries.iloc[16,2] = 'TWG'
top30_countries.iloc[18,2] = 'KOR'

ModuleNotFoundError: No module named 'get_data'

In [ ]:
wto_export[wto_export['Reporter'].str.match('taiwan')]

Since there is no data about Taiwan in WTO dataset, we omit Taiwan from top30 countries.

In [ ]:
top30_countries_wo_taiwan = top30_countries.drop(index = 16)
wto_exp_pivoted = wto_export.pivot_table(index = ['ReporterISO3A', 'Reporter'], columns='Year', values='Value')
wtoexp_top30_wo_twn = wto_exp_pivoted.loc[top30_countries_wo_taiwan['iso3a']]
wtoexp_top30_wo_twn['growth'] = (wtoexp_top30_wo_twn[2020] / wtoexp_top30_wo_twn[2019] - 1) * 100
wtoexp_top30_wo_twn_high = wtoexp_top30_wo_twn.sort_values(["growth"], ascending= False).head(10)
wtoexp_top30_wo_twn_low = wtoexp_top30_wo_twn.sort_values(["growth"]).head(10)

display(wtoexp_top30_wo_twn_low)
display(wtoexp_top30_wo_twn_high)


In [ ]:
# Bar Chart
wtoexp_top30_wo_twn_low.reset_index(inplace=True)
low_bar = px.bar(wtoexp_top30_wo_twn_low, x = "Reporter", y = "growth")
low_bar.show()

In [ ]:
wtoexp_top30_wo_twn_high.reset_index(inplace=True)
high_bar = px.bar(wtoexp_top30_wo_twn_high, x = "Reporter", y = "growth")
high_bar.show()

## Second Question: Correlation and Linear Regression

Covid-19 had a considerable impact to the economy in 2020, particularly in the international trade sector.
The following graph compare the trade values for major exporters in the world in 2019 and 2020.
We can see, obviously, that most countries was affected by the adverse impact of covid-19 crisis.


Moreover, if we plot the scatterplot between the number of covid patients and trade value in 2020 for each country

Furthermore, the government policy during the pandemic (i.e. lockdown) is one of the key source of decrease in trade values. 
The government policy caused some firms to be temporarily shut down, while many employees have to work at their home as much as possible.
As a result, the export sector encountered labor shortages in every process, from manufacturing to logistics. Sometimes, we called as a supply-chain crisis.
That means many economic activities had to suspend. 

The University of Oxford released the stringency index, which measures the level of "tightening" in the government policies during the pandemic.
We hypothesised that if countries imposed policies more tighter, the level of export values value should lower as the business activities had to temporarily shut down.

In [ ]:
# Set up the data for this question
owid_df = 
comtrade_df = 

### Set up OWID 

In [ ]:
un_comtrade.head()


In [ ]:
# Group By

agg = {'total_cases': 'last', 'new_cases': 'sum', 
       'stringency_index': 'mean'}
columns = ['yr', 'rtTitle', 'TradeValue']
un_q1 = un_comtrade[columns]

grouped = un_comtrade.groupby(['rt3ISO', 'yr']).sum().reset_index()
un_q1 = grouped[grouped['yr'] == 2020]
un_q1.head()


In [ ]:
agg = {'total_cases': 'last', 'new_cases': 'sum', 
       'stringency_index': 'mean'}
columns = ['yr', 'rtTitle', 'TradeValue']
un_q1_2 = un_comtrade[columns]

un_q1_2_df = un_q1_2.groupby(['rt3ISO', 'yr']).sum().unstack()
un_q1_2_df['growth'] = un_q1_2_df['2020'] / un_q1_2_df['2019'] - 1 * 100

un_q1_2_df.head()

In [ ]:
owid_df.head()

In [ ]:
q1_df = un_q1.merge(owid_df, how='inner', left_on = 'rt3ISO', right_on = 'iso_code')

In [ ]:
q1_df.shape

In [ ]:
scatter = px.scatter(q1_df, x="total_cases", y="TradeValue", color="rt3ISO",
                        size='TradeValue')
scatter.show()

In [ ]:
scatter = px.scatter(q1_df, x="stringency_index", y="TradeValue", color="rt3ISO",
                        size='TradeValue')
scatter.show()